# Full Prophet Model on Complete Dataset (No Downsampling)
This notebook is intended to run on a **dedicated server** with sufficient memory. It loads a full-resolution dataset and uses all features of Prophet (trend, seasonalities, holidays, changepoints) to make minute-level forecasts.

In [ ]:
!pip install prophet openpyxl

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from prophet import Prophet
from sklearn.metrics import mean_absolute_error, mean_squared_error
import joblib
import os

SAVE_DIR = "prophet_output"
os.makedirs(SAVE_DIR, exist_ok=True)

# Load full-resolution dataset
df = pd.read_excel("usask.sec.min_short_v2.xlsx")
df.columns = ['minute', 'requests']
df['minute'] = pd.to_datetime(df['minute'], unit='m', origin='unix')
df.rename(columns={'minute': 'ds', 'requests': 'y'}, inplace=True)


In [ ]:
plt.figure(figsize=(14, 5))
plt.plot(df['ds'], df['y'], label='Requests per minute')
plt.xlabel("Time")
plt.ylabel("Requests")
plt.title("Original Time Series - Minute Resolution")
plt.grid(True)
plt.legend()
plt.savefig(f"{SAVE_DIR}/full_timeseries_plot.png")
plt.show()


In [ ]:
model = Prophet(
    growth='linear',
    yearly_seasonality=True,
    weekly_seasonality=True,
    daily_seasonality=True,
    changepoint_prior_scale=0.05,
    seasonality_prior_scale=10.0,
    holidays_prior_scale=10.0,
    n_changepoints=25
)

# Optional: Add hourly seasonality (or others if applicable)
model.add_seasonality(name='hourly', period=1, fourier_order=3)


In [ ]:
model.fit(df)
joblib.dump(model, f"{SAVE_DIR}/full_model.joblib")


In [ ]:
# Forecast next 30 minutes
future = model.make_future_dataframe(periods=30, freq='min')
forecast = model.predict(future)

forecast.to_csv(f"{SAVE_DIR}/full_forecast.csv", index=False)


In [ ]:
fig1 = model.plot(forecast)
fig1.savefig(f"{SAVE_DIR}/full_forecast_plot.png")

fig2 = model.plot_components(forecast)
fig2.savefig(f"{SAVE_DIR}/full_components_plot.png")


In [ ]:
# Holdout evaluation (last 1 day = 1440 minutes)
train = df[:-1440]
test = df[-1440:]

model_eval = Prophet(
    growth='linear',
    yearly_seasonality=True,
    weekly_seasonality=True,
    daily_seasonality=True,
    changepoint_prior_scale=0.05,
    seasonality_prior_scale=10.0,
    holidays_prior_scale=10.0,
    n_changepoints=25
)
model_eval.add_seasonality(name='hourly', period=1, fourier_order=3)
model_eval.fit(train)
future_eval = model_eval.make_future_dataframe(periods=1440, freq='min')
forecast_eval = model_eval.predict(future_eval)

# Merge and compute RMSE
merged = forecast_eval[['ds', 'yhat']].merge(test, on='ds')
rmse = np.sqrt(mean_squared_error(merged['y'], merged['yhat']))
mae = mean_absolute_error(merged['y'], merged['yhat'])

with open(f"{SAVE_DIR}/evaluation.txt", "w") as f:
    f.write(f"MAE: {mae:.2f}\nRMSE: {rmse:.2f}\n")

print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")


In [ ]:
from sklearn.metrics import r2_score, mean_absolute_percentage_error

def evaluate_predictions(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    mape = mean_absolute_percentage_error(y_true, y_pred)
    accuracy = max(0, 100 * (1 - mape))  # Clip negative accuracy to 0%
    return mae, rmse, r2, mape, accuracy


In [ ]:
import matplotlib.pyplot as plt

# Predict next 1 minute from multiple windows in the last 90 minutes
windows = [
    ('Window 1 (90 min)', -90),
    ('Window 2 (60 min)', -60),
    ('Window 3 (45 min)', -45),
    ('Window 4 (30 min)', -30),
    ('Window 5 (15 min)', -15)
]

results = []

for label, idx in windows:
    hist = df[:idx]
    test = df[idx:idx+1]

    temp_model = Prophet(
        growth='linear',
        yearly_seasonality=True,
        weekly_seasonality=True,
        daily_seasonality=True,
        changepoint_prior_scale=0.05,
        seasonality_prior_scale=10.0,
        holidays_prior_scale=10.0,
        n_changepoints=25
    )
    temp_model.add_seasonality(name='hourly', period=1, fourier_order=3)
    temp_model.fit(hist)

    future = temp_model.make_future_dataframe(periods=1, freq='min')
    forecast = temp_model.predict(future)

    # Merge and evaluate
    merged = forecast[['ds', 'yhat']].merge(test, on='ds')
    mae, rmse, r2, mape, accuracy = evaluate_predictions(merged['y'], merged['yhat'])
    results.append((label, mae, rmse, r2, mape, accuracy))

    # Plot
    plt.figure(figsize=(10, 3))
    plt.plot(merged['ds'], merged['y'], label='Actual', marker='o')
    plt.plot(merged['ds'], merged['yhat'], label='Forecast', marker='x')
    plt.title(f'{label}: 1-Minute Forecast')
    plt.xlabel("Time")
    plt.ylabel("Requests")
    plt.legend()
    plt.grid(True)
    plt.savefig(f"{SAVE_DIR}/forecast_{label.replace(' ', '_')}.png")
    plt.show()


In [ ]:
import pandas as pd
result_df = pd.DataFrame(results, columns=['Window', 'MAE', 'RMSE', 'R2', 'MAPE', 'Accuracy (%)'])
result_df.to_csv(f"{SAVE_DIR}/multi_window_forecast_summary.csv", index=False)
result_df
